In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/uber_rides_data.xlsx - sample_train.csv')

What is the shape of given dataset?


In [3]:
data


,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...
199995,42598914,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695416,1


What is the data type of ' pickup_datetime' feature in your data?

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


What is the average fare amount?
*
Remove the null values from the dataframe to answer the following question.

In [5]:
data['fare_amount'].mean()

11.359955250000002

In [6]:
data['dropoff_longitude'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 200000 entries, 0 to 199999
Series name: dropoff_longitude
Non-Null Count   Dtype  
--------------   -----  
199999 non-null  float64
dtypes: float64(1)
memory usage: 1.5 MB


Calculate distance between each pickup and dropoff points using Haversine formula.
What is the median haversine distance between pickup and dropoff location according to the given dataset?

In [23]:
# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

# Calculate Haversine distances for each record
data['haversine_distance'] = data.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

# Calculate the median Haversine distance
median_distance = data['haversine_distance'].median()
print("Median Haversine Distance:", median_distance, "kilometers")

Median Haversine Distance: 2.1209923961833708 kilometers


What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [24]:
data['haversine_distance'].max()

16409.239135313168

How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?

In [25]:

count_zero_distance_rides = len(data[data['haversine_distance'] == 0.0])

print(f'{count_zero_distance_rides} rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset')

5632 rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset


What is the mean 'fare_amount' for rides with 0 haversine distance?
*
Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

In [26]:
mean_fare_amount_zero_distance = data[data['haversine_distance'] == 0.0]['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine distance:", mean_fare_amount_zero_distance)

Mean 'fare_amount' for rides with 0.0 Haversine distance: 11.585317826704546


What is the maximum 'fare_amount' for a ride?

In [27]:
max_amt = data['fare_amount'].max()
print(f'Max fare amount : {max_amt}')

Max fare amount : 499.0


What is the haversine distance between pickup and dropoff location for the costliest ride?
*
Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

In [28]:
costliest_ride = data.sort_values(by='fare_amount', ascending=False).iloc[0]

pickup_coords = (costliest_ride['pickup_latitude'], costliest_ride['pickup_longitude'])
dropoff_coords = (costliest_ride['dropoff_latitude'], costliest_ride['dropoff_longitude'])


haversine_distance = haversine(*pickup_coords, *dropoff_coords)

print("Haversine Distance for Costliest Ride:", haversine_distance, "kilometers")


Haversine Distance for Costliest Ride: 0.0007899213191009993 kilometers


How many rides were recorded in the year 2014?


In [36]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])

In [37]:
rides_2014 = data[data['pickup_datetime'].dt.year==2014]

print("Number of rides in the year 2014:", len(rides_2014))

Number of rides in the year 2014: 29968


How many rides were recorded in the first quarter of 2014?


In [38]:
rides_1st_q = data[(data['pickup_datetime'] >= '2014-1-1') & (data['pickup_datetime'] <= '2014-04-1')]

print("Number of rides in the 1st quarter of year 2014:", len(rides_1st_q))


Number of rides in the 1st quarter of year 2014: 7687


On which day of the week in September 2010, maximum rides were recorded ?

In [39]:
data['ride_week_day'] = data['pickup_datetime'].dt.day_name()

In [40]:
rides_september_2010 = data[(data['pickup_datetime'] >= '2010-09-01') & (data['pickup_datetime'] <= '2010-10-1')]


day_of_week_counts = rides_september_2010['ride_week_day'].value_counts()


max_day = day_of_week_counts.idxmax()
max_count = day_of_week_counts[max_day]

print(f"On {max_day} in September 2010, the maximum number of rides were recorded: {max_count} rides")


On Thursday in September 2010, the maximum number of rides were recorded: 457 rides


Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

Which algorithm gives the least adjusted R square value?


In [41]:
data = data.dropna()
data.isna().sum()


ride_id               0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
haversine_distance    0
ride_week_day         0
dtype: int64

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,RobustScaler,StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor


In [43]:
# Split the data into features (X) and the target variable (y)
cont_col = ['passenger_count', 'haversine_distance']
nom_col = [ 'ride_week_day']

x = data[cont_col+nom_col]
y = data['fare_amount']

# Perform a 70-30 split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Preprocessing the data

preprocessor = ColumnTransformer(transformers = [
            ('cont_pipeline' , Pipeline([
               ( 'cont_scale', StandardScaler())
            ]), cont_col),

            ('nom_pipeline', Pipeline([
                ('nom_encode',OneHotEncoder())
            ]),nom_col)
],remainder = 'passthrough')

x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)


In [44]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regression': RandomForestRegressor(),
    'KNN Regressor': KNeighborsRegressor(),
    'Decission Tree Regressor':DecisionTreeRegressor()
}

adjusted_r2_values = {}

for model_name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = x_test.shape[1]

    adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    adjusted_r2_values[model_name] = adjusted_r2

# Find the algorithm with the least adjusted R-squared value
least_adjusted_r2_algorithm = min(adjusted_r2_values, key=adjusted_r2_values.get)

print("Adjusted R-squared values:")
for model_name, adjusted_r2 in adjusted_r2_values.items():
    print(f"{model_name}: {adjusted_r2}")

print("Algorithm with the least adjusted R-squared value:", least_adjusted_r2_algorithm)





Adjusted R-squared values:
Linear Regression: 0.0005964405874608891
Random Forest Regression: 0.624289323840069
KNN Regressor: 0.6262358292465445
Decission Tree Regressor: 0.47881335346548815
Algorithm with the least adjusted R-squared value: Linear Regression


Conclusion:
Linear regression model gives the least adjusted R square value.